In [1]:
import json     #Fetching the externally sourced data
new_data = []
with open('news_data.json',encoding="utf8") as rf:   
    data = json.load(rf)
for i in data:
    new_data.append(i['text'])

In [2]:
new_list = []    #Fetching the Train data set and making two list of text and labels
train_labels = []
count = 0
with open("train.json") as json_file:
        data = json.load(json_file)
for key,value in data.items():
    new_list.append(data[key]["text"])
    train_labels.append(data[key]["label"])

In [3]:
label = 0    # Adding the externally sourced data to the train files
for i in range(0,1200):
    new_list.append(new_data[i])
    train_labels.append(label)

In [4]:
dev_list = []   # Fetching the Dev data set
dev_labels = []
with open("C:\\Users\\hp\\Desktop\\Uni\\Sem 3\\NLP\\Assignment 2\\project-files\\project-files\\dev.json") as json_file:
        data = json.load(json_file)
for key,value in data.items():
    dev_list.append(data[key]["text"])
    dev_labels.append(data[key]["label"])

In [5]:
import nltk   # Preprocessing using NLTK package, to clean and filter the data sets
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

porter = PorterStemmer()

tokenizer = nltk.RegexpTokenizer(r"\w+")  # Using Regex tokenizer to remove special characters from the text

stopwords = set(stopwords.words('english'))
train_tweets = []
dev_tweets = []
for i in range(0,len(new_list)):
    new_list[i] = new_list[i].lower()
    prepro = tokenizer.tokenize(new_list[i])
    preproStopWords = [w for w in prepro if not w in stopwords]
    filtered_sentence = (" ").join(preproStopWords)
    train_tweets.append(filtered_sentence)
prepro,preproStopWords,stemmed = [],[],[]

for i in range(0,len(dev_list)):
    dev_list[i] = dev_list[i].lower()
    prepro = tokenizer.tokenize(dev_list[i])
    preproStopWords = [w for w in prepro if not w in stopwords]
    filtered_sentence = (" ").join(preproStopWords)
    dev_tweets.append(filtered_sentence)

In [6]:
from sklearn.feature_extraction.text import HashingVectorizer

vectorizer = HashingVectorizer(n_features=10000000,norm='l1')
X_train = vectorizer.fit_transform(train_tweets)
X_dev = vectorizer.transform(dev_tweets)

In [7]:
from sklearn import tree  # Implementing Decision Tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, train_labels)
pred = clf.predict(X_dev)

print(accuracy_score(pred,dev_labels))

p, r, f, _ = precision_recall_fscore_support(dev_labels, pred, pos_label=1, average="binary")

print("Performance on the positive class (documents with misinformation):")
print("Precision =", p)
print("Recall    =", r)
print("F1        =", f)

0.73
Performance on the positive class (documents with misinformation):
Precision = 0.6716417910447762
Recall    = 0.9
F1        = 0.7692307692307693


In [8]:
import nltk     # Again preprocessing, as data types different for CountVectorizer and HashingVectorizer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from collections import defaultdict

tokenizer = nltk.RegexpTokenizer(r"\w+")
stopwords = set(stopwords.words('english'))

X_train = []
for i in range(0,len(new_list)):
    new_list[i] = new_list[i].lower()
    prepro = tokenizer.tokenize(new_list[i])
    preproStopWords = [w for w in prepro if not w in stopwords]
    #stemmed = [porter.stem(word) for word in preproStopWords]
    filtered_sentence = (" ").join(preproStopWords)
    X_train.append(filtered_sentence)
X_dev = []
for i in range(0,len(dev_list)):
    dev_list[i] = dev_list[i].lower()
    prepro = tokenizer.tokenize(dev_list[i])
    preproStopWords = [w for w in prepro if not w in stopwords]
    #stemmed = [porter.stem(word) for word in preproStopWords]
    filtered_sentence = (" ").join(preproStopWords)
    X_dev.append(filtered_sentence)

In [9]:
from sklearn.pipeline import Pipeline  # Implementing LinearSVC model 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC


sgd = Pipeline([('vect', CountVectorizer()),
                ('clf', LinearSVC(random_state=0, tol=1e-5,max_iter= 5000)),
               ])
sgd.fit(X_train, train_labels)
y_pred = sgd.predict(X_dev)

print(accuracy_score(y_pred,dev_labels))

q, w, e, _ = precision_recall_fscore_support(dev_labels, y_pred, pos_label=1, average="binary")

print("Performance on the positive class (documents with misinformation):")
print("Precision =", q)
print("Recall    =", w)
print("F1        =", e)

0.77
Performance on the positive class (documents with misinformation):
Precision = 0.7014925373134329
Recall    = 0.94
F1        = 0.8034188034188033


In [72]:
test_list = []       # Fetching the test data 
with open("C:\\Users\\hp\\Desktop\\Uni\\Sem 3\\NLP\\Assignment 2\\project-files\\project-files\\test-unlabelled.json") as json_file:
        data = json.load(json_file)
for key,value in data.items():
    test_list.append(data[key]["text"])

In [ ]:
test_tweets = []     # Cleaning the test data set

for i in range(0,len(test_list)):
    test_list[i] = test_list[i].lower()
    prepro = tokenizer.tokenize(test_list[i])
    preproStopWords = [w for w in prepro if not w in stopwords]
    filtered_sentence = (" ").join(preproStopWords)
    test_tweets.append(filtered_sentence)

In [76]:
y_test = sgd.predict(test_list)  # Getting predictions for test data set

In [82]:
results = {}   # Saving the test results
for i in range(0,len(y_test)):
    results["test-"+str(i)] = {"label":int(y_test[i])}
with open('C:\\Users\\hp\\Desktop\\Uni\\Sem 3\\NLP\\Assignment 2\\project-files\\project-files\\New Text Document.json', 'w') as outfile:
    json.dump(results,outfile)